In [1]:
%%capture

!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece
!pip install -U bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
import pandas as pd
from datasets import Dataset


In [3]:
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `prova` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `prova`


# Preprocessing dei dati

In [4]:
!git clone https://github.com/benedettoscala/ifttt-code-generator
%cd ifttt-code-generator/
!git pull

Cloning into 'ifttt-code-generator'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 23 (delta 5), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 25.20 KiB | 460.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.
Filtering content: 100% (4/4), 171.23 MiB | 32.45 MiB/s, done.
/content/ifttt-code-generator
Already up to date.


In [5]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import BitsAndBytesConfig

#Configurazione modello base e quantizzazione
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

#Caricamento del modello e tokenizer
print("Caricamento del modello base...")
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("Caricamento del tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Prepara il modello per k-bit training (disabilita gradienti su pesi int4 ecc.)
model = prepare_model_for_kbit_training(model)

# 3. Configurazione LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
    # i moduli dei transformer li ho controllati su hugging face, sono questi (godo)
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


Caricamento del modello base...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [ ]:
# Caricamento e pulizia del dataset
csv_path = "combined_and_cleaned.csv"
df = pd.read_csv(csv_path)

df.dropna(subset=["cleaned_description", "filter_code"], inplace=True)

df.drop_duplicates(subset=["cleaned_description","filter_code"], inplace=True)

#Suddivisione train   e val
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset  = Dataset.from_pandas(eval_df)

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["cleaned_description"],
        text_target=examples["filter_code"],
        truncation=True,
        max_length=512,
        padding="max_length"  # se vuoi padding dinamico, sostituisci con "longest"
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset  = eval_dataset.map(tokenize_function,  batched=True)


# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, pad_to_multiple_of=8)

In [ ]:
#Impostazioni di training
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,                     # aumentato a 3 epoche
    per_device_train_batch_size=2,          # batch per GPU
    gradient_accumulation_steps=4,          # accumula gradienti -> eff. batch = 8
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",             # scheduler "cosine"
    learning_rate=1e-4,                     # ridotto rispetto a 2e-4
    weight_decay=0.01,                      # alzato a 0.01
    max_grad_norm=1.0,                      # grad clip standard
    save_steps=100,                         # salva meno frequentemente
    logging_steps=10,                       # log ogni 10 step
    evaluation_strategy="steps",            # valuta ogni X step
    eval_steps=5,                         # ogni step fai eval
    load_best_model_at_end=True,            # carica modello migliore
    save_total_limit=3,                     # mantieni solo 3 checkpoint
    fp16=False,                             # se la tua GPU non supporta BF16, puoi abilitare fp16
    bf16=False,                             # se la tua GPU supporta BF16, puoi metterlo a True
    report_to="none"                        # o "tensorboard", "wandb", ...
)


def compute_metrics(eval_pred):
    return {}

#Creazione Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

#Salvataggio finale LoRA + quantization
trainer.save_model("./results/best_model")

print("Fine del training!")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 1. Specifica il modello base e la cartella dove hai salvato il fine-tuning
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
FINETUNED_MODEL_DIR = "./results/best_model"

# 2. Carica il tokenizer del modello base
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token  # assicurati che il token di padding sia quello di fine sequenza

# 3. Carica il modello base quantizzato (4 bit) o in bfloat16 (a seconda di come hai fatto l'addestramento)
#    Se l'hai addestrato con quantization a 4 bit, userai la stessa configurazione di BitsAndBytes
#    che hai usato in precedenza.
#    Per semplicità, qui mostriamo una versione "standard" (fp16) o bf16, da adattare se necessario.
model_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",         # mappa automaticamente su GPU/CPU
    torch_dtype=torch.float16,  # usa float16; se la tua GPU supporta bf16, puoi mettere torch.bfloat16
)

# 4. Carica i pesi LoRA salvati (PeftModel)
model = PeftModel.from_pretrained(
    model_base,
    FINETUNED_MODEL_DIR,
    torch_dtype=torch.float16   # o bfloat16, in coerenza con quanto fatto su base_model
)

# Metti il modello in eval
model.eval()

# 5. Funzione di generazione di codice (o di testo)
def generate_code(prompt, max_new_tokens=128):
    """
    prompt: testo di input (descrizione, istruzioni, ecc.)
    max_new_tokens: numero massimo di token di output
    """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # sposta su GPU
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,         # se vuoi campionare
            top_p=0.9,              # top-p sampling
            temperature=0.8,        # regola la "creatività"
            repetition_penalty=1.1  # leggero penalty su ripetizioni
        )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)


test_prompt = (
    "if the current hour is 17, send a tweet"
)
generated = generate_code(test_prompt, max_new_tokens=100)
print("PROMPT:\n", test_prompt)
print("\nCODICE GENERATO:\n", generated)


In [ ]:
# Percorso del modello base e configurazione
base_model = "mistralai/Mistral-7B-Instruct-v0.2"  # Sostituisci con il tuo modello preferito
new_model = "filter_code_model_lora"

# Configura BitsAndBytes per quantizzazione a 4 bit
bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Carica il modello e il tokenizer
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Prepara il modello per l'addestramento con k-bit
model = prepare_model_for_kbit_training(model)

# Configura LoRA
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

# Applica LoRA al modello
model = get_peft_model(model, lora_config)

# Mostra i parametri addestrabili
model.print_trainable_parameters()


In [ ]:
# Caricamento del dataset
train_path = "combined.csv"  # Percorso al dataset di training
df = pd.read_csv(train_path)
# Prepara il dataset
df['text'] = df['cleaned_description'] + " </s>" + df['filter_code']  # Combina input e output

df = df.dropna(subset=['cleaned_description', 'filter_code'])

df.isnull().sum()
dataset = Dataset.from_pandas(df)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Suddividi il dataset in training e valutazione
train_data, eval_data = train_test_split(df, test_size=0.2, random_state=42)

# Trasforma in dataset Hugging Face
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

# Tokenizza i dataset
def tokenize_function(examples):
    return tokenizer(
        examples["cleaned_description"],
        text_target=examples["filter_code"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


In [ ]:
# Tokenizza il dataset
def tokenize_function(example):
    return tokenizer(
        example['cleaned_description'],
        text_target=example['filter_code'],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import Dataset
from sklearn.model_selection import train_test_split
# Configura i parametri di addestramento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,                     # aumentato a 3 epoche
    per_device_train_batch_size=2,          # batch per GPU
    gradient_accumulation_steps=4,          # accumula gradienti -> eff. batch = 8
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",             # scheduler "cosine"
    learning_rate=1e-4,                     # ridotto rispetto a 2e-4
    weight_decay=0.01,                      # alzato a 0.01
    max_grad_norm=1.0,                      # grad clip standard
    save_steps=100,                         # salva meno frequentemente
    logging_steps=10,                       # log ogni 10 step
    evaluation_strategy="no",               # nessuna valutazione automatica
    load_best_model_at_end=False,           # non necessario se non valuti
    save_total_limit=3,                     # mantieni solo 3 checkpoint
    fp16=False,                             # se la tua GPU non supporta BF16, puoi abilitare fp16
    bf16=False,                             # se la tua GPU supporta BF16, puoi metterlo a True
    report_to="wandb"                        # o "tensorboard", "wandb", ...
)


# Usa un data collator per gestire padding dinamico
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Configura il Trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Dataset di valutazione obbligatorio
    data_collator=data_collator,  # Usa il data collator al posto del tokenizer
)


In [ ]:
# Addestramento
trainer.train()


In [ ]:
# Salva il modello e il tokenizer
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)


In [ ]:
# Carica il modello fine-tunato
model = AutoModelForCausalLM.from_pretrained(new_model)
tokenizer = AutoTokenizer.from_pretrained(new_model)

# Genera un filter code
description = "Log calendar events to Google Sheets."
inputs = tokenizer(description, return_tensors="pt", truncation=True, max_length=512).to("cuda")
outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4)
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Description:", description)
print("Generated Filter Code:", generated_code)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configura la quantizzazione a 4 bit
bnb_config = BitsAndBytesConfig(
    bnb_4bit_quant_type="nf4",             # Tipo di quantizzazione
    bnb_4bit_compute_dtype=torch.bfloat16,  # Precisione di calcolo
    bnb_4bit_use_double_quant=False        # Usa o meno la doppia quantizzazione
)

# Specifica la directory del modello salvato
saved_model_directory = "filter_code_model_lora"  # Sostituisci con il percorso del tuo modello

# Carica il modello con quantizzazione a 4 bit
model = AutoModelForCausalLM.from_pretrained(
    saved_model_directory,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",  # Assegna automaticamente il modello ai dispositivi disponibili
)

# Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained(saved_model_directory)

# Sposta il modello sul dispositivo appropriato (GPU o CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


In [ ]:
# Esempio di input per il modello
description = "Send an email when a new file is added to Dropbox"

# Tokenizza l'input
inputs = tokenizer(description, return_tensors="pt", truncation=True, max_length=512).to(device)

# Genera un output con il modello
outputs = model.generate(
    inputs["input_ids"],
    max_length=512,
    num_beams=4,         # Usa la ricerca a raggio per risultati migliori
    early_stopping=True  # Ferma la generazione quando raggiunge un punto
)

# Decodifica il risultato generato
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Stampa il risultato
print("Description:", description)
print("Generated Filter Code:", generated_code)